# AI Chat Stylist Implementation & Testing Walkthrough

This notebook demonstrates how to configure, run, and validate the **AI Chat Stylist** project. It covers environment preparation, instantiating the stylist with or without live OpenAI access, adding optional outfit imagery, and executing automated tests.


## 1. Environment Preparation

Install the project dependencies. When running inside this repository's root directory the command below will reuse the shared `requirements.txt` file. Uncomment the cell if you have not already installed the dependencies in your environment.


In [ ]:
# %pip install -r ../requirements.txt


## 2. Configure Credentials

Set the `OPENAI_API_KEY` environment variable if you plan to call the live OpenAI APIs. Without a key, the notebook automatically falls back to lightweight stub implementations so you can still explore the flow offline.


In [ ]:
import os

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
if OPENAI_API_KEY:
    print("✅ OPENAI_API_KEY detected — live API calls enabled.")
else:
    print("⚠️ OPENAI_API_KEY not found. The notebook will run in offline demo mode.")


## 3. Import Project Modules

Add the repository root to `sys.path` so the `ai_chat_stylist` package can be imported, then bring in the key classes. The helper below automatically swaps in stubbed dependencies when the API key is unavailable.


In [ ]:
import sys
from pathlib import Path

REPO_ROOT = Path('..').resolve()
if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))

from ai_chat_stylist.chatbot import AIChatStylist, StylistConfig
from ai_chat_stylist import chatbot as chatbot_module


### Helper: Build a Stylist Instance

`build_notebook_stylist` returns an `AIChatStylist` configured for either live usage (when an API key exists) or deterministic offline experimentation. In offline mode it:

- Substitutes a simple language model stub that produces consistent responses.
- Replaces the outfit image description function with a quick rule-based summary.
- Seeds the vector store with basic wardrobe guidance for retrieval demos.


In [ ]:
from typing import Optional
from langchain_core.messages import AIMessage

class OfflineLLM:
    """Deterministic fallback LLM for environments without OpenAI access."""

    def invoke(self, messages):
        human_turns = [getattr(m, 'content', '') for m in messages if getattr(m, 'type', '') == 'human']
        prompt = human_turns[-1] if human_turns else "your wardrobe"
        return AIMessage(content=(
            "Offline demo response: consider layering a textured cardigan with "
            f"{prompt.lower()} and add an accent accessory for balance."
        ))

def build_notebook_stylist(system_prompt: Optional[str] = None) -> AIChatStylist:
    config = StylistConfig()
    if system_prompt:
        config.system_prompt = system_prompt

    if OPENAI_API_KEY:
        return AIChatStylist(config=config)

    def offline_llm_factory(cfg):
        return OfflineLLM()

    def offline_embedding_factory(cfg):
        class DummyEmbeddings:
            def embed_documents(self, texts):
                return [[float(idx + 1)] * 3 for idx, _ in enumerate(texts)]
            def embed_query(self, text):
                return [0.0, 0.0, 0.0]
        return DummyEmbeddings()

    from langchain_community.vectorstores import FAISS

    stylist = AIChatStylist(
        config=config,
        llm_factory=offline_llm_factory,
        embedding_factory=offline_embedding_factory,
        vectorstore=FAISS.from_texts(
            [
                "The user prefers smart-casual looks with neutral colors.",
                "They own white sneakers, dark denim, and a camel trench coat.",
            ],
            embedding=offline_embedding_factory(config.vector_config),
        ),
    )

    def offline_describe_outfit_image(image_path: str, prompt: Optional[str] = None) -> str:
        return (
            "Offline summary for '{image}': neutral palette with relaxed tailoring."
        ).format(image=Path(image_path).name)

    chatbot_module.describe_outfit_image = offline_describe_outfit_image
    return stylist


## 4. Instantiate the Stylist

Use the helper to create the stylist instance. Adjust the optional prompt to experiment with different personalities.


In [ ]:
stylist = build_notebook_stylist(
    system_prompt="You are a cheerful fashion expert who emphasizes comfort and versatility."
)
stylist


## 5. (Optional) Add Outfit Imagery

Provide the path to an outfit photograph to enrich the stylist's memory. In offline mode, the notebook produces a deterministic textual summary so you can verify the ingestion flow without external services.


In [ ]:
# Replace with your own image path if available.
example_image = Path('../tests/fixtures/example_outfit.jpg')
if example_image.exists():
    description = stylist.add_outfit_image(str(example_image))
    print('Image summary:', description)
else:
    print('No example image found. Skipping image ingestion step.')


## 6. Chat with the Stylist

Enter a message to test the full conversational loop. Each turn is stored in the FAISS-backed memory so subsequent interactions can reference prior wardrobe details.


In [ ]:
user_question = "I'm meeting friends for brunch — what outfit should I build?"
response = stylist.chat(user_question)
print('Stylist response:
', response)


### Inspect Stored Context

Review the texts persisted to the vector store to confirm that conversations and image summaries are captured for future retrieval.


In [ ]:
for idx, context in enumerate(stylist.export_context(), start=1):
    print(f"{idx:02d}: {context}")


## 7. Run Automated Tests

Execute the repository's pytest suite directly from the notebook to validate the LangChain pipeline, retrieval behavior, and image ingestion flow.


In [ ]:
!pytest -q
